In [1]:
import unicodedata
import os
import re
from tqdm import tqdm
from collections import Counter

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
BASE_DIR = '/content/drive/MyDrive/проект'

# Stage 2 (NFC -> NFD)

In [4]:
def convert_to_nfd_force(files_map, base_dir=BASE_DIR):
    """
    Принудительная конвертация содержимого файлов в NFD.
    Разлагает составные символы (например, 'й' -> 'и' + '̆').
    """
    for input_file, output_file in files_map.items():
        input_path = os.path.join(base_dir, input_file)
        output_path = os.path.join(base_dir, output_file)

        if not os.path.exists(input_path):
            print(f"Файл не найден: {input_path}")
            continue

        # 1. Читаем файл целиком
        with open(input_path, 'r', encoding='utf-8') as f_in:
            content = f_in.read()

        # 2. ЖЕСТКО переводим в NFD
        nfd_content = unicodedata.normalize('NFD', content)

        # 3. Записываем результат
        with open(output_path, 'w', encoding='utf-8') as f_out:
            f_out.write(nfd_content)

        print(f"NFD конвертация завершена: {input_file} -> {output_file}")

In [5]:
tasks = {
    # Adyghe
    '1_Adyghe_OCR.txt': '2_Adyghe_OCR.txt',
    '1_Adyghe_NoOCR.txt': '2_Adyghe_NoOCR.txt',

    # Kabardian
    '1_Kabardian_OCR.txt': '2_Kabardian_OCR.txt',
    '1_Kabardian_NoOCR.txt': '2_Kabardian_NoOCR.txt',
}
convert_to_nfd_force(tasks)

NFD конвертация завершена: 1_Adyghe_OCR.txt -> 2_Adyghe_OCR.txt
NFD конвертация завершена: 1_Adyghe_NoOCR.txt -> 2_Adyghe_NoOCR.txt
NFD конвертация завершена: 1_Kabardian_OCR.txt -> 2_Kabardian_OCR.txt
NFD конвертация завершена: 1_Kabardian_NoOCR.txt -> 2_Kabardian_NoOCR.txt


# Stage 3 (Normalize sticks)

In [6]:
def analyze_and_fix_sticks(input_file, base_dir=BASE_DIR):
    """
    Нормализует палочки (1 l | ! Ӏ → I)
    ТОЛЬКО в словах, состоящих из кириллицы + палочек.
    Считает статистику ДО и ПОСЛЕ и сохраняет её в файл.
    """

    input_path = os.path.join(base_dir, input_file)
    if not os.path.exists(input_path):
        print(f"Файл не найден: {input_path}")
        return

    with open(input_path, 'r', encoding='utf-8') as f:
        text = f.read()

    # --- СТРОГАЯ проверка слова ---
    def is_cyrillic_word(word):
        sticks = {'1', 'l', '|', '!', 'Ӏ', 'I'}
        for ch in word:
            if ch in sticks:
                continue
            if 'CYRILLIC' not in unicodedata.name(ch, ''):
                return False
        return True

    sticks = {'1', 'l', '|', '!', 'Ӏ', 'I'}

    # --- Функция подсчёта палочек ---
    def count_sticks(text):
        stats = {k: 0 for k in sticks}
        for word in re.findall(r'\w+', text, flags=re.UNICODE):
            if not is_cyrillic_word(word):
                continue
            for i, ch in enumerate(word):
                if ch not in stats:
                    continue
                if ch == '!':
                    if i == len(word) - 1:
                        continue  # ! справа не считаем
                stats[ch] += 1
        return stats

    # ---------- ДО ----------
    stats_before = count_sticks(text)

    # ---------- ЗАМЕНА ----------
    def replace(match):
        word = match.group()
        if not is_cyrillic_word(word):
            return word

        result = []
        for i, ch in enumerate(word):
            if ch in {'1', 'l', '|', 'Ӏ'}:
                result.append('I')
            elif ch == '!':
                if i == len(word) - 1:
                    result.append(ch)
                else:
                    result.append('I')
            else:
                result.append(ch)
        return ''.join(result)

    text_after = re.sub(r'\w+', replace, text, flags=re.UNICODE)

    # ---------- ПОСЛЕ ----------
    stats_after = count_sticks(text_after)

    # ---------- ВЫВОД И СОХРАНЕНИЕ СТАТИСТИКИ ----------
    stats_filename = f"sticks_stats_{input_file}.txt"
    stats_path = os.path.join(base_dir, stats_filename)

    total_before = sum(stats_before.values())
    total_after = sum(stats_after.values())

    with open(stats_path, 'w', encoding='utf-8') as f:
        f.write("СТАТИСТИКА ПАЛОЧЕК ДО → ПОСЛЕ\n")
        f.write("=" * 40 + "\n")
        for k in ['I', '1', 'l', '|', '!', 'Ӏ']:
            b = stats_before.get(k, 0)
            a = stats_after.get(k, 0)
            f.write(f"{k:>2}: {b:,} → {a:,}  ({a - b:+,})\n")
        f.write("-" * 40 + "\n")
        f.write(f"ВСЕГО: {total_before:,} → {total_after:,}\n")

    print(f"\nСТАТИСТИКА сохранена: {stats_filename}")
    print("\nДетали:")
    for k in ['I', '1', 'l', '|', '!', 'Ӏ']:
        b = stats_before.get(k, 0)
        a = stats_after.get(k, 0)
        print(f"{k:>2}: {b:,} → {a:,}  ({a - b:+,})")
    print(f"ВСЕГО: {total_before:,} → {total_after:,}")

    # ---------- СОХРАНЕНИЕ ИСПРАВЛЕННОГО ФАЙЛА ----------
    base_name = '_'.join(input_file.split('_')[1:])
    output_file = f"3_{base_name}"
    with open(os.path.join(base_dir, output_file), 'w', encoding='utf-8') as f:
        f.write(text_after)

    print(f"\nФайл с исправленным текстом сохранён: {output_file}")

In [7]:
# Adyghe
analyze_and_fix_sticks('2_Adyghe_OCR.txt')
analyze_and_fix_sticks('2_Adyghe_NoOCR.txt')

# Kabardian
analyze_and_fix_sticks('2_Kabardian_OCR.txt')
analyze_and_fix_sticks('2_Kabardian_NoOCR.txt')


СТАТИСТИКА сохранена: sticks_stats_2_Adyghe_OCR.txt.txt

Детали:
 I: 3,877,778 → 3,898,158  (+20,380)
 1: 20,370 → 0  (-20,370)
 l: 10 → 0  (-10)
 |: 0 → 0  (+0)
 !: 0 → 0  (+0)
 Ӏ: 0 → 0  (+0)
ВСЕГО: 3,898,158 → 3,898,158

Файл с исправленным текстом сохранён: 3_Adyghe_OCR.txt

СТАТИСТИКА сохранена: sticks_stats_2_Adyghe_NoOCR.txt.txt

Детали:
 I: 3,847,882 → 3,868,197  (+20,315)
 1: 20,305 → 0  (-20,305)
 l: 10 → 0  (-10)
 |: 0 → 0  (+0)
 !: 0 → 0  (+0)
 Ӏ: 0 → 0  (+0)
ВСЕГО: 3,868,197 → 3,868,197

Файл с исправленным текстом сохранён: 3_Adyghe_NoOCR.txt

СТАТИСТИКА сохранена: sticks_stats_2_Kabardian_OCR.txt.txt

Детали:
 I: 7,883,186 → 10,402,528  (+2,519,342)
 1: 177,210 → 0  (-177,210)
 l: 44,139 → 0  (-44,139)
 |: 0 → 0  (+0)
 !: 0 → 0  (+0)
 Ӏ: 2,297,993 → 0  (-2,297,993)
ВСЕГО: 10,402,528 → 10,402,528

Файл с исправленным текстом сохранён: 3_Kabardian_OCR.txt

СТАТИСТИКА сохранена: sticks_stats_2_Kabardian_NoOCR.txt.txt

Детали:
 I: 6,634,330 → 6,991,419  (+357,089)
 1: 129,0

# Stage 4

## Проверка на римские цифры

In [8]:
# латинские римские цифры
latin_roman = set("IVXLCDM")
# кириллические похожие
cyr_roman = set("ІХСМ")

# римское число (любое сочетание)
roman_pattern = re.compile(r'(?<!\w)[IVXLCDMІХСМ]+(?!\w)')

def analyze_file(input_file, output_file, base_dir=BASE_DIR):
    """Анализ римских цифр в файле"""
    clean = {}
    mixed = {}
    cyrillic_only = {}

    input_path = os.path.join(base_dir, input_file)
    output_path = os.path.join(base_dir, output_file)

    if not os.path.exists(input_path):
        print(f"Нет такого файла: {input_path}")
        return

    with open(input_path, 'r', encoding='utf-8') as f:
        text = f.read()

    matches = roman_pattern.findall(text)

    for word in matches:
        has_latin = any(ch in latin_roman for ch in word)
        has_cyr = any(ch in cyr_roman for ch in word)

        if has_latin and not has_cyr:
            clean[word] = clean.get(word, 0) + 1
        elif has_latin and has_cyr:
            mixed[word] = mixed.get(word, 0) + 1
        elif has_cyr and not has_latin:
            cyrillic_only[word] = cyrillic_only.get(word, 0) + 1

    clean_total = sum(clean.values())
    mixed_total = sum(mixed.values())
    cyr_total = sum(cyrillic_only.values())
    total = clean_total + mixed_total + cyr_total

    if total > 0:
        clean_pct = clean_total / total * 100
        mixed_pct = mixed_total / total * 100
        cyr_pct = cyr_total / total * 100
    else:
        clean_pct = mixed_pct = cyr_pct = 0

    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(f"Файл: {input_file}\n\n")

        f.write("СТАТИСТИКА\n")
        f.write("-" * 30 + "\n")
        f.write(f"Чистые (латиница): {clean_total} ({clean_pct:.5f}%)\n")
        f.write(f"Смешанные: {mixed_total} ({mixed_pct:.5f}%)\n")
        f.write(f"Только кириллица: {cyr_total} ({cyr_pct:.5f}%)\n\n")

        f.write("ЧИСТЫЕ РИМСКИЕ ЦИФРЫ (латиница)\n")
        f.write("-" * 30 + "\n")
        for k, v in clean.items():
            f.write(f"{k} : {v}\n")

        f.write("\nСМЕШАННЫЕ РИМСКИЕ ЦИФРЫ\n")
        f.write("-" * 30 + "\n")
        for k, v in mixed.items():
            f.write(f"{k} : {v}\n")

        f.write("\nТОЛЬКО КИРИЛЛИЧЕСКИЕ\n")
        f.write("-" * 30 + "\n")
        for k, v in cyrillic_only.items():
            f.write(f"{k} : {v}\n")

    print(f"Анализ сохранён: {output_file}")

In [9]:
# Adyghe
analyze_file('3_Adyghe_OCR.txt', 'roman_stats_4_Adyghe_OCR.txt')
analyze_file('3_Adyghe_NoOCR.txt', 'roman_stats_4_Adyghe_NoOCR.txt')

# Kabardian
analyze_file('3_Kabardian_OCR.txt', 'roman_stats_4_Kabardian_OCR.txt')
analyze_file('3_Kabardian_NoOCR.txt', 'roman_stats_4_Kabardian_NoOCR.txt')

Анализ сохранён: roman_stats_4_Adyghe_OCR.txt
Анализ сохранён: roman_stats_4_Adyghe_NoOCR.txt
Анализ сохранён: roman_stats_4_Kabardian_OCR.txt
Анализ сохранён: roman_stats_4_Kabardian_NoOCR.txt


## Приводим римские цифры к латинице

In [10]:
# замена кириллических символов на латинские
cyr_to_lat = {
    'І': 'I',
    'Х': 'X',
    'С': 'C',
    'М': 'M'
}

roman_pattern = re.compile(r'(?<!\w)[IVXLCDMІХСМ]+(?!\w)')

def normalize_roman(match):
    """Замена кириллических символов на латинские в римских цифрах"""
    word = match.group()
    result = ""
    for ch in word:
        result += cyr_to_lat.get(ch, ch)
    return result

def normalize_file(input_file, output_file, base_dir=BASE_DIR):
    """Нормализация римских цифр в файле"""
    input_path = os.path.join(base_dir, input_file)
    output_path = os.path.join(base_dir, output_file)

    if not os.path.exists(input_path):
        print(f"Файл не найден: {input_path}")
        return

    with open(input_path, 'r', encoding='utf-8') as f:
        text = f.read()

    new_text = roman_pattern.sub(normalize_roman, text)

    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(new_text)

    print(f"Нормализация завершена: {input_file} -> {output_file}")

In [11]:
# Adyghe
normalize_file('3_Adyghe_OCR.txt', '4_Adyghe_OCR.txt')
normalize_file('3_Adyghe_NoOCR.txt', '4_Adyghe_NoOCR.txt')

# Kabardian
normalize_file('3_Kabardian_OCR.txt', '4_Kabardian_OCR.txt')
normalize_file('3_Kabardian_NoOCR.txt', '4_Kabardian_NoOCR.txt')

Нормализация завершена: 3_Adyghe_OCR.txt -> 4_Adyghe_OCR.txt
Нормализация завершена: 3_Adyghe_NoOCR.txt -> 4_Adyghe_NoOCR.txt
Нормализация завершена: 3_Kabardian_OCR.txt -> 4_Kabardian_OCR.txt
Нормализация завершена: 3_Kabardian_NoOCR.txt -> 4_Kabardian_NoOCR.txt


## Считаем смешанные слова (кириллица + латиница) и чисто кириллические + I.

In [12]:
def calculate_mixed_percentage(file_path, output_suffix='', base_dir=BASE_DIR):
    """Подсчет смешанных слов (кириллица + латиница кроме I)"""
    full_path = os.path.join(base_dir, file_path)

    if not os.path.exists(full_path):
        print(f"Файл не найден: {full_path}")
        return

    try:
        with open(full_path, 'r', encoding='utf-8') as f:
            text = f.read()
    except Exception as e:
        print(f"Ошибка при чтении {file_path}: {e}")
        return

    allowed_latin = {'I'}
    words = re.findall(r'\w+', text)

    pure_cyrillic_words = []
    mixed_words = []

    for word in words:
        if len(word) <= 1:
            continue

        has_cyrillic = False
        has_forbidden_latin = False

        for char in word:
            u_name = unicodedata.name(char, '')

            if 'CYRILLIC' in u_name:
                has_cyrillic = True
            elif 'LATIN' in u_name:
                if char not in allowed_latin:
                    has_forbidden_latin = True

        if has_cyrillic:
            if has_forbidden_latin:
                mixed_words.append(word)
            else:
                pure_cyrillic_words.append(word)

    pure_cyrillic_count = len(pure_cyrillic_words)
    mixed_count = len(mixed_words)

    # Сохраняем список смешанных слов
    output_file = os.path.join(base_dir, f'mixed_words{output_suffix}.txt')
    try:
        with open(output_file, 'w', encoding='utf-8') as f:
            for word in mixed_words:
                f.write(word + '\n')
        print(f"✓ Смешанные слова сохранены в: mixed_words{output_suffix}.txt")
    except Exception as e:
        print(f"Ошибка при сохранении: {e}")

    # Формируем статистику
    stats_text = [
        f"\n=== СТАТИСТИКА ДЛЯ {file_path} ===",
        f"(Кириллица + I): {pure_cyrillic_count}",
        f"Смешанных (Кириллица + иная латиница): {mixed_count}"
    ]

    total = pure_cyrillic_count + mixed_count
    if total > 0:
        stats_text.append(f"Процент «грязных» слов: {(mixed_count / total) * 100:.5f}%")

    if mixed_words:
        stats_text.append("\n=== ПРИМЕРЫ СМЕШАННЫХ (первые 10) ===")
        stats_text.extend(mixed_words[:10])

    stats_text.append("\n" + "="*60)

    # Вывод в консоль
    print("\n".join(stats_text))

    # Сохранение статистики
    stats_file = os.path.join(base_dir, f'stats{output_suffix}.txt')
    try:
        with open(stats_file, 'w', encoding='utf-8') as f:
            f.write("\n".join(stats_text))
        print(f"Статистика сохранена в: stats{output_suffix}.txt")
    except Exception as e:
        print(f"Ошибка при сохранении статистики: {e}")

In [13]:
# Adyghe
calculate_mixed_percentage('4_Adyghe_OCR.txt', '_4_Adyghe_OCR')
calculate_mixed_percentage('4_Adyghe_NoOCR.txt', '_4_Adyghe_NoOCR')

# Kabardian
calculate_mixed_percentage('4_Kabardian_OCR.txt', '_4_Kabardian_OCR')
calculate_mixed_percentage('4_Kabardian_NoOCR.txt', '_4_Kabardian_NoOCR')

✓ Смешанные слова сохранены в: mixed_words_4_Adyghe_OCR.txt

=== СТАТИСТИКА ДЛЯ 4_Adyghe_OCR.txt ===
(Кириллица + I): 10690039
Смешанных (Кириллица + иная латиница): 387
Процент «грязных» слов: 0.00362%

=== ПРИМЕРЫ СМЕШАННЫХ (первые 10) ===
Аҧсшəа
АDZAHA
адыгэa
адзыхэa
адыгэa
адыгэa
Кuzey
Aхэмэ
Mоу
OгушIуа

Статистика сохранена в: stats_4_Adyghe_OCR.txt
✓ Смешанные слова сохранены в: mixed_words_4_Adyghe_NoOCR.txt

=== СТАТИСТИКА ДЛЯ 4_Adyghe_NoOCR.txt ===
(Кириллица + I): 10607046
Смешанных (Кириллица + иная латиница): 386
Процент «грязных» слов: 0.00364%

=== ПРИМЕРЫ СМЕШАННЫХ (первые 10) ===
Аҧсшəа
АDZAHA
адыгэa
адзыхэa
адыгэa
адыгэa
Кuzey
Aхэмэ
Mоу
OгушIуа

Статистика сохранена в: stats_4_Adyghe_NoOCR.txt
✓ Смешанные слова сохранены в: mixed_words_4_Kabardian_OCR.txt

=== СТАТИСТИКА ДЛЯ 4_Kabardian_OCR.txt ===
(Кириллица + I): 30389408
Смешанных (Кириллица + иная латиница): 142339
Процент «грязных» слов: 0.46620%

=== ПРИМЕРЫ СМЕШАННЫХ (первые 10) ===
cделать
иcпеченного
термоc
Пi

## Удаление смешанных слов

In [14]:
def remove_mixed_words_from_file(original_file, mixed_words_file, output_file, base_dir=BASE_DIR):
    """
    Быстро удаляет смешанные слова из файла (построчно + set)
    """
    original_path = os.path.join(base_dir, original_file)
    mixed_path = os.path.join(base_dir, mixed_words_file)
    output_path = os.path.join(base_dir, output_file)

    # Проверяем, что файлы существуют
    if not os.path.exists(original_path):
        print(f"Оригинальный файл '{original_file}' не найден!")
        return
    if not os.path.exists(mixed_path):
        print(f"Файл со списком смешанных слов '{mixed_words_file}' не найден!")
        return

    # Читаем список смешанных слов
    print(f"Читаем {mixed_words_file}...")
    try:
        with open(mixed_path, 'r', encoding='utf-8') as f:
            mixed_words = set(line.strip() for line in f if line.strip())
        print(f"Уникальных слов для удаления: {len(mixed_words)}")
    except Exception as e:
        print(f"Ошибка при чтении {mixed_words_file}: {e}")
        return

    removed_total = 0
    line_count = 0

    print(f"Читаем и очищаем {original_file} построчно...")
    try:
        with open(original_path, 'r', encoding='utf-8') as infile, \
             open(output_path, 'w', encoding='utf-8') as outfile:

            for line in tqdm(infile, desc="Обработка строк"):
                line_count += 1

                # Находим все слова (учитывая кириллицу, латиницу и цифры)
                words = re.findall(r'\w+', line, flags=re.UNICODE)

                # Собираем новую строку без "грязных" слов
                new_line = ""
                pos = 0
                for word in words:
                    start = line.find(word, pos)
                    end = start + len(word)
                    # добавляем все символы между предыдущим словом и текущим (пробелы, знаки)
                    new_line += line[pos:start]
                    if word not in mixed_words:
                        new_line += word
                    else:
                        removed_total += 1
                    pos = end
                new_line += line[pos:]  # добавляем остаток строки
                outfile.write(new_line)

        print(f"Очищенный текст сохранён в {output_file}")
        print(f"Всего строк обработано: {line_count}")
        print(f"Удалено слов: {removed_total}\n")

    except Exception as e:
        print(f"Ошибка при обработке {original_file}: {e}")

In [15]:
# Adyghe
remove_mixed_words_from_file(
    '4_Adyghe_OCR.txt',
    'mixed_words_4_Adyghe_OCR.txt',
    '4_Adyghe_OCR_final.txt'
)
remove_mixed_words_from_file(
    '4_Adyghe_NoOCR.txt',
    'mixed_words_4_Adyghe_NoOCR.txt',
    '4_Adyghe_NoOCR_final.txt'
)

# Kabardian
remove_mixed_words_from_file(
    '4_Kabardian_OCR.txt',
    'mixed_words_4_Kabardian_OCR.txt',
    '4_Kabardian_OCR_final.txt'
)
remove_mixed_words_from_file(
    '4_Kabardian_NoOCR.txt',
    'mixed_words_4_Kabardian_NoOCR.txt',
    '4_Kabardian_NoOCR_final.txt'
)

Читаем mixed_words_4_Adyghe_OCR.txt...
Уникальных слов для удаления: 293
Читаем и очищаем 4_Adyghe_OCR.txt построчно...


Обработка строк: 1it [00:10, 10.09s/it]


Очищенный текст сохранён в 4_Adyghe_OCR_final.txt
Всего строк обработано: 1
Удалено слов: 387

Читаем mixed_words_4_Adyghe_NoOCR.txt...
Уникальных слов для удаления: 292
Читаем и очищаем 4_Adyghe_NoOCR.txt построчно...


Обработка строк: 1it [00:09,  9.96s/it]


Очищенный текст сохранён в 4_Adyghe_NoOCR_final.txt
Всего строк обработано: 1
Удалено слов: 386

Читаем mixed_words_4_Kabardian_OCR.txt...
Уникальных слов для удаления: 77075
Читаем и очищаем 4_Kabardian_OCR.txt построчно...


Обработка строк: 1it [00:34, 34.24s/it]


Очищенный текст сохранён в 4_Kabardian_OCR_final.txt
Всего строк обработано: 1
Удалено слов: 142339

Читаем mixed_words_4_Kabardian_NoOCR.txt...
Уникальных слов для удаления: 4164
Читаем и очищаем 4_Kabardian_NoOCR.txt построчно...


Обработка строк: 1it [00:55, 55.53s/it]


Очищенный текст сохранён в 4_Kabardian_NoOCR_final.txt
Всего строк обработано: 1
Удалено слов: 5733

